<a href="https://colab.research.google.com/github/anatorres09/DE-MRI_Classificacio_CNN/blob/main/entrenament_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Connectar a Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
#Importar llibreries
import os
from glob import glob
import cv2
import nibabel as nib
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
import gc
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
from joblib import Parallel, delayed

In [7]:
drive_folder = '/content/drive/MyDrive/CNN'

In [11]:
#CNN IMATGES .PNG

# Ruta de la carpeta principal
ruta_carpeta_normal = os.path.join(drive_folder, "corr_entrenament", "corr_normal")
ruta_carpeta_patologic = os.path.join(drive_folder, "corr_entrenament", "corr_patologic")
# Estrategia 1: Redimensionar las imágenes a un tamaño fijo
resize_shape = (128, 128)

# Recoger imágenes y etiquetas de la carpeta corr_normal
def recoger_imagenes_etiquetas(ruta_carpeta, etiqueta, resize_shape):
    casos = [nom_carpeta for nom_carpeta in os.listdir(ruta_carpeta) if os.path.isdir(os.path.join(ruta_carpeta, nom_carpeta))]

    imatges = []
    etiquetes = []

    for cas in casos:
        ruta_cas = os.path.join(ruta_carpeta, cas)
        rutes_imatges = glob(os.path.join(ruta_cas, '*.png'))

        for ruta_imatge in rutes_imatges:
            imatge = cv2.imread(ruta_imatge)
            imatge = cv2.cvtColor(imatge, cv2.COLOR_BGR2RGB)
            
            # Aplicar la estrategia 1: Redimensionar las imágenes
            imatge = cv2.resize(imatge, resize_shape)
            
            imatge = imatge.astype('float32') / 255.0  # Convertir a float32 y normalizar entre 0 y 1
            imatges.append(imatge)
            etiquetes.append(etiqueta)

    return imatges, etiquetes



# Recollir imatges i etiquetes de la carpeta corr_normal
imatges_normal, etiquetes_normal = recollir_imatges_etiquetes(ruta_carpeta_normal, "normal", resize_shape)

# Recollir imatges i etiquetes de la carpeta corr_patologic
imatges_patologic, etiquetes_patologic = recollir_imatges_etiquetes(ruta_carpeta_patologic, "patologic", resize_shape)

# Concatenar les llistes de imatges i etiquetes
imatges_total = imatges_normal + imatges_patologic
etiquetes_total = etiquetes_normal + etiquetes_patologic

# Convertir la lista de imágenes y etiquetas en una matriz numpy
matriu_imatges = np.array(imatges_total, dtype='float32')
etiquetes = np.array(etiquetes_total)

print(f"S'han recollit {matriu_imatges.shape[0]} imatges en total.")


# Obtenir la quantitat de imatges etiquetades com "normal"
quantitat_normal = np.count_nonzero(etiquetes == "normal")

# Obtenir la quntitat de imatges etiquetades com "patologic"
quantitat_patologic = np.count_nonzero(etiquetes == "patologic")

print(f"S'han trobat {quantitat_normal} imatges etiquetades com 'normal'.")
print(f"S'han trobat {quantitat_patologic} imatges etiquetades com 'patologic'.")

S'han recollit 699 imatges en total.
S'han trobat 226 imatges etiquetades com 'normal'.
S'han trobat 473 imatges etiquetades com 'patologic'.


In [12]:

# Preprocessat de les imatges
# Normalitzar els valors de píxels al rang [0, 1]
imatges = matriu_imatges/ 255.0

# Dividir les dades en conjunts d'entrenament i prova
X_train, X_test, y_train, y_test = train_test_split(imatges, etiquetes, test_size=0.2, random_state=42)



# Initialize LabelEncoder
label_encoder = LabelEncoder()

# Fit and transform labels
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)

# Convert encoded labels to one-hot encoding
y_train = tf.one_hot(y_train_encoded, depth=2)
y_test = tf.one_hot(y_test_encoded, depth=2)

# Construir el model de la CNN
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=X_train[0].shape),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(2, activation='softmax')
])

# Compilar el model
model.compile(optimizer='adam',
               loss='categorical_crossentropy',
               metrics=['accuracy'])

# Crear generadores de datos para el entrenamiento y la prueba
generador_entrenamiento = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255, validation_split=0.3, dtype='float16')
generador_prueba = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255, validation_split=0.3, dtype='float16')

# Crear generadores de flujo de datos para el entrenamiento y la prueba
flujo_entrenamiento = generador_entrenamiento.flow(X_train, y_train, batch_size=32)
flujo_prueba = generador_prueba.flow(X_test, y_test, batch_size=32)


# Antes de entrenar el modelo
gc.collect()  # Liberar la memoria antes del entrenamiento

# Entrenar el modelo utilizando generadores de datos
history = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))


#Ruta del archivo donde se guardará el modelo
ruta_model = os.path.join(drive_folder, "model_CNN.h5")

# Guardar el modelo
model.save(ruta_model)
print("Model de la CNN guardat correctament.")

# Evaluar el modelo en el conjunto de prueba utilizando generador de datos
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Perdua en el conjunt de prova: {loss}")
print(f"Precisió en el conjunt de prova: {accuracy}")

# Obtener las predicciones del conjunto de prueba
y_pred = model.predict(X_test)

# Convertir las predicciones en etiquetas codificadas
y_pred_encoded = np.argmax(y_pred, axis=1)
y_test_encoded = np.argmax(y_test, axis=1)

# Calcular y mostrar la matriz de confusión
confusion_mat = confusion_matrix(y_test_encoded, y_pred_encoded)
print("Matriz de confusión:")
print(confusion_mat)

# Calcular y mostrar el informe de clasificación
classification_rep = classification_report(y_test_encoded, y_pred_encoded)
print("Informe de clasificación:")
print(classification_rep)

# Calcular y mostrar la precisión
precision = precision_score(y_test_encoded, y_pred_encoded)
print(f"Precisión: {precision}")

# Calcular y mostrar el recall
recall = recall_score(y_test_encoded, y_pred_encoded)
print(f"Recall: {recall}")

# Calcular y mostrar el F1-score
f1 = f1_score(y_test_encoded, y_pred_encoded)
print(f"F1-score: {f1}")

# Calcular y mostrar la exactitud (accuracy)
accuracy = accuracy_score(y_test_encoded, y_pred_encoded)
print(f"Exactitud: {accuracy}")

# Liberar la memoria después de evaluar el modelo
del X_train, y_train, X_test, y_test
gc.collect()  # Liberar la memoria utilizada por las variables eliminadas

Epoch 1/10
18/18 [==============================] - 2s 31ms/step - loss: 0.6316 - accuracy: 0.6619 - val_loss: 0.6655 - val_accuracy: 0.6143
Epoch 2/10
18/18 [==============================] - 0s 20ms/step - loss: 0.6243 - accuracy: 0.6923 - val_loss: 0.6860 - val_accuracy: 0.6143
Epoch 3/10
18/18 [==============================] - 0s 19ms/step - loss: 0.6118 - accuracy: 0.6923 - val_loss: 0.6620 - val_accuracy: 0.6143
Epoch 4/10
18/18 [==============================] - 0s 19ms/step - loss: 0.6031 - accuracy: 0.6923 - val_loss: 0.6331 - val_accuracy: 0.6143
Epoch 5/10
18/18 [==============================] - 0s 17ms/step - loss: 0.5682 - accuracy: 0.6923 - val_loss: 0.5996 - val_accuracy: 0.6143
Epoch 6/10
18/18 [==============================] - 0s 17ms/step - loss: 0.5455 - accuracy: 0.6923 - val_loss: 0.5503 - val_accuracy: 0.6143
Epoch 7/10
18/18 [==============================] - 0s 17ms/step - loss: 0.5266 - accuracy: 0.7084 - val_loss: 0.5543 - val_accuracy: 0.6929
Epoch 8/10
18

4067